# JIT Parameters
Author:Jørgen S. Dokken

In this chapter, we will explore how to optimize and inspect the integration kernels used in DOLFINx.
As we have seen in the previous demos, dolfinx uses the [Unified form language](https://github.com/FEniCS/ufl/) to describe variational problems.

These descriptions has to be translated in to code for assembling the right and left hand side of the discrete variational problem. 

DOLFINx uses [ffcx](https://github.com/FEniCS/ffcx/) to generate efficient C code assembling the element matrices.
This C code is in turned compiled using [CFFI](https://cffi.readthedocs.io/en/latest/), and we can specify a variety of compile options.

We start by specifying the current directory as the place to place the generated C files, we obtain the current directory using pathlib

In [1]:
from pathlib import Path
cache_dir = f"{str(Path.cwd())}/.cache"
print(f"Directory to put C files in: {cache_dir}")

Directory to put C files in: /home/shared/chapter4/.cache


Next we generate a general function to assemble the mass matrix for a UnitCube. Note that we use `dolfinx.fem.Form` to compile the variational form. For codes using `dolfinx.LinearProblem`, you can supply `jit_parameters` as a keyword argument.

In [2]:
import dolfinx
from mpi4py import MPI
from typing import Dict
import time
import ufl

def compile_form(space:str, degree:int, jit_parameters:Dict):
    N = 10
    mesh = dolfinx.UnitCubeMesh(MPI.COMM_WORLD, N, N, N)
    V = dolfinx.FunctionSpace(mesh, (space, degree))
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.inner(u, v) * ufl.dx
    a_compiled = dolfinx.fem.Form(a, jit_parameters=jit_parameters)
    start = time.time()
    dolfinx.fem.assemble_matrix(a_compiled)
    end = time.time()
    print(f"({space}, {degree}) {jit_parameters['cffi_extra_compile_args']}: {end-start} s")

We start by considering the different levels of optimization the C compiled can use on the optimized code. A list of optimization options and explainations can be found [here](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html)

In [3]:
optimization_options = ["-O1", "-O2", "-O3", "-Ofast"]

The next option we can choose is if we want to compile the code with `-march=native` or not. This option enables instructions for the local machine, and can give different results on different systems. More information can be found [here](https://gcc.gnu.org/onlinedocs/gcc/AArch64-Options.html#g_t-march-and--mcpu-Feature-Modifiers)

In [4]:
march_native = [True, False]

We choose a subset of finite element spaces, varying the order of the space to look at the effects it has on the assembly time with different compile options.

In [5]:
for space in ["N1curl", "CG", "RT"]:
    for degree in [1, 2, 3]:
        for native in march_native:
            for option in optimization_options:
                if native:
                    cffi_options = [option, "-march=native"]
                else:
                    cffi_options = [option]
                jit_parameters = {"cffi_extra_compile_args": cffi_options, 
                    "cache_dir": cache_dir, "cffi_libraries": ["m"]}
                compile_form(space, degree, jit_parameters=jit_parameters)

(N1curl, 1) ['-O1', '-march=native']: 0.02358245849609375 s
(N1curl, 1) ['-O2', '-march=native']: 0.02239680290222168 s
(N1curl, 1) ['-O3', '-march=native']: 0.019269227981567383 s
(N1curl, 1) ['-Ofast', '-march=native']: 0.01844191551208496 s
(N1curl, 1) ['-O1']: 0.02351093292236328 s
(N1curl, 1) ['-O2']: 0.022663593292236328 s
(N1curl, 1) ['-O3']: 0.0205535888671875 s
(N1curl, 1) ['-Ofast']: 0.01865553855895996 s
(N1curl, 2) ['-O1', '-march=native']: 0.36463403701782227 s
(N1curl, 2) ['-O2', '-march=native']: 0.28248047828674316 s
(N1curl, 2) ['-O3', '-march=native']: 0.15567302703857422 s
(N1curl, 2) ['-Ofast', '-march=native']: 0.17487168312072754 s
(N1curl, 2) ['-O1']: 0.34765076637268066 s
(N1curl, 2) ['-O2']: 0.2997701168060303 s
(N1curl, 2) ['-O3']: 0.18477249145507812 s
(N1curl, 2) ['-Ofast']: 0.17911696434020996 s
(N1curl, 3) ['-O1', '-march=native']: 2.34515118598938 s
(N1curl, 3) ['-O2', '-march=native']: 2.121826171875 s
(N1curl, 3) ['-O3', '-march=native']: 0.887540340423

As we observed in the output, the main influence on the compilation time is the optimization level, but in certain cases, `-march=native` can reduce the compilation file significantly.